In [23]:
# parameters
import pickle
import com_func
import pandas as pd

def dummy(doc):
    return doc

threshold = 10
cutoff = 3
Dataset = "pubmed"
# engineer file
coauthor_emb_type = "off"
venue_emb_type = "tf"
year_emb_type = "tf"
pp_textual_emb_type = "off"
citation_emb_type = "off"


In [24]:
# read trained rec to rec tf-idf textual graph
def read_textual_embedding(Dataset = "pubmed", emb_type = "off"):
    textual_emb = []
    while True:
        if emb_type == "lsa":
            loadDir = "../Data/"+Dataset+"/models/LSA/textual/"
            with open(loadDir+'lsa_d100_emb.pickle', "rb") as input_file:
                emb = pickle.load(input_file)
            with open(loadDir+'feature_pid.pickle', "rb") as input_file:
                pid = pickle.load(input_file)
                
            print(emb.shape)
            print(len(pid))
            break
        elif emb_type == "pv_dm":
            loadDir = "../Data/"+Dataset+"/vectors/d2v/Doc2Vec(dmm,d100,n5,w5,mc3,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    paper_Vectors = read_data
                    textual_emb.append(paper_Vectors)
            f.close()

            print("Total vector records:",len(textual_emb))
            print(textual_emb[0])
            break
        elif emb_type == "pv_dbow":
            loadDir = "../Data/"+Dataset+"/vectors/d2v/Doc2Vec(dbow,d100,n5,mc3,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    paper_Vectors = read_data
                    textual_emb.append(paper_Vectors)
            f.close()
            
            print("Total vector records:",len(textual_emb))
            print(textual_emb[0])
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="pv_dbow"
    return textual_emb

In [25]:
# read trained rec to rec node2vec citation graph
def read_citation_embedding(Dataset = "pubmed", emb_type = "off"):
    citation_emb = []
    while True:
        if emb_type == "n2v":
            citation_emb_dir = "../Data/"+Dataset+"/vectors/"+emb_type+"/n2v.txt"
            with open(citation_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    if(len(read_data)==101):
                        paper_Vectors = read_data
                        citation_emb.append(paper_Vectors)
            f.close()
            print("Total vector records:",len(citation_emb))
            print(citation_emb[:3])
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="n2v"
    return citation_emb

In [26]:
def read_labeled_file(infile):
    LabeledRecords_original = []
    with open(infile, 'r', encoding = 'utf8') as f:
        for line in f:
            read_data = line.split("\t")
            # get ride of bad formated lines
            if(len(read_data)==13 or len(read_data)==12):
                paper_detail = {"paperID": read_data[0], "authorID":read_data[1], 
                                "total_author": read_data[3], "co-author": read_data[5],
                                "venue_id": read_data[7], "publish_year": read_data[10]}
                LabeledRecords_original.append(paper_detail)
            else:
                print(len(read_data))
        f.close()
    return pd.DataFrame(LabeledRecords_original)

In [27]:
# co-author relation to emb
def co_author_to_vector(raw_co_author_data, emb_type="off"):
    while True:
        if emb_type == "tf":
            co_author_vectorizer = CountVectorizer()
            print(co_author_vectorizer)
            result_vector = co_author_vectorizer.fit_transform(raw_co_author_data).toarray()
            #print(co_author_vectorizer.get_feature_names())
            #print(len(co_author_vectorizer.vocabulary_))
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="tf"
    return result_vector

In [28]:
# author-venue relation to emb
def venue_to_vector(raw_venue_id, emb_type="off"):
    while True:
        if emb_type == "tf":
            count_vectorizer = CountVectorizer()
            result_vector = count_vectorizer.fit_transform(raw_venue_id).toarray()
            #print(len(count_vectorizer.vocabulary_))
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="tf"
    return result_vector

In [29]:
# author-year relation to emb
def year_to_vector(raw_year, emb_type="off"):
    while True:
        if emb_type == "tf":
            count_vectorizer = CountVectorizer()
            result_vector = count_vectorizer.fit_transform(raw_year).toarray()
            #print(len(count_vectorizer.vocabulary_))
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="tf"
    return result_vector

In [30]:
def extract_embedding(all_embedding, pid):
    extracted_emb = []
    wanted_pid = pid.values.tolist()
    # only if embedding exist
    if len(all_embedding)>0:
        for paper_embedding in all_embedding:
            if paper_embedding[0] in wanted_pid:
                extracted_emb.append(paper_embedding)
    
    extracted_emb = pd.DataFrame(extracted_emb)
    # only if embedding exist
    if len(all_embedding)>0:
        # reorder embedding with pid and fill empty record with 0
        extracted_emb = pd.merge(pid.to_frame(), extracted_emb, left_on='paperID', right_on=0, how='outer')
        # remove index
        extracted_emb.drop(['paperID', 0], axis=1, inplace=True)
    return extracted_emb

In [31]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score
# cross validation
def k_fold_cv(data, label, clf, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for train_index, test_index in kf.split(data):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        # fit data to clf
        clf.fit(data_train, label_train)
        # get predicted label
        label_pred = clf.predict(data_test)
        allTrueLabel.extend(label_test)
        allPredLabel.extend(label_pred)
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='macro')
    
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    
    return accuracy, f1

In [32]:
# read pretrained embeddings
all_textual_embedding = read_textual_embedding(Dataset, pp_textual_emb_type)
all_citation_embedding = read_citation_embedding(Dataset, citation_emb_type)
print(len(all_textual_embedding))
print(len(all_citation_embedding))

0
0


In [34]:
# load the file
import sys
import io
import os
import collections
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)
# 
clf_type ="mnb"
# collect statistic to output
allname = []
num_class = []
per_class_count = []

all_svcLinear_accuracy = []
all_svcLinear_f1 = []
all_LR_accuracy = []
all_LR_f1 = []


# read all file in labeled group
for file in listfiles:
    # group name
    temp = file.split("_")
    name = temp[1]+"_"+temp[-1]
    print("For name: ",name)
    allname.append(name)
    # read needed content in labeled file
    labeled_data_part = read_labeled_file(fileDir+file)
    print("total sample size before apply threshold: ",len(labeled_data_part))
    # count number of paper each author write based on author ID
    paperCounter = collections.Counter(labeled_data_part["authorID"])
    print(paperCounter)
    # collect per class statistic
    for k in list(paperCounter):
        if paperCounter[k] < threshold:
            del paperCounter[k]
    temp =list(paperCounter.keys())
    print(temp)
    per_class_count.append(paperCounter)
    num_class.append(len(paperCounter))
    # remove samples that are smaller than threshold
    labeled_data_part = labeled_data_part[labeled_data_part.authorID.isin(temp)]
    print("Total sample size after apply threshold: ",len(labeled_data_part))
    # if only have one class or no class pass the threshold, not applicable
    if(len(paperCounter)==0) or (len(paperCounter)==1):
        all_svcLinear_accuracy.append("Not applicable")
        all_svcLinear_f1.append("Not applicable")
        all_LR_accuracy.append("Not applicable")
        all_LR_f1.append("Not applicable")
    else:
        # convert author id to label
        gather_label = []
        for index, record in labeled_data_part.iterrows():
            gather_label.append(temp.index(record["authorID"]))
        labeled_data_part["label"] = gather_label
        # shuffle the data
        labeled_data = labeled_data_part.sample(frac=1).reset_index(drop=True)
        # extract true label and it's corresponeding pid for matching
        label = labeled_data["label"]
        pid = labeled_data["paperID"]
        reorder_idx = labeled_data.index
        # list of different data field
        part_collection = []
        # select feature wanted to fit to clustering/classification algorithm
        # data part 1, co-author matrix
        data_part_co_author = co_author_to_vector(labeled_data["co-author"], emb_type=coauthor_emb_type)
        print(data_part_co_author.shape)
        part_collection.append(data_part_co_author)
        # data part 2.1, venue_id that author attend
        data_part_venue = venue_to_vector(labeled_data["venue_id"], emb_type=venue_emb_type)
        print(data_part_venue.shape)
        part_collection.append(data_part_venue)
        # data part 2.2 year that author attend
        data_part_year = year_to_vector(labeled_data["publish_year"], emb_type=year_emb_type)
        print(data_part_year.shape)
        part_collection.append(data_part_year)
        # data part 3.1, extract textual embedding
        data_part_textual = extract_embedding(all_textual_embedding, pid)
        print(data_part_textual.shape)
        part_collection.append(data_part_textual)
        # data part 4, read citation embedding 
        data_part_citation = extract_embedding(all_citation_embedding, pid)
        print(data_part_citation.shape)
        part_collection.append(data_part_citation)
        # merge different part of data data together by concatenate it all together
        # remove empty emb (when emb set off)
        part_collection = [part for part in part_collection if len(part)!=0]
        print(len(part_collection))
        if len(part_collection)>1:
            combinedata = np.concatenate(part_collection,axis=1)
        elif len(part_collection)==1:
            if isinstance(part_collection[0], pd.DataFrame):
                combinedata = part_collection[0].values
            else:
                combinedata = part_collection[0]
        else:
            print("No data available")
            break
        print(combinedata.shape)
        # using converted feature vector to train classifier
        # using SVM with linear kernal
        # use 10 fold cv
        clf = SVC(decision_function_shape='ovr', kernel='linear')
        svcaccuracy, svcmarcof1 = k_fold_cv(combinedata, label, clf, k=10)
        print("svc Accuracy: ",svcaccuracy)
        print("svc F1: ", svcmarcof1)
        all_svcLinear_accuracy.append(svcaccuracy)
        all_svcLinear_f1.append(svcmarcof1)
        # using logistic regression
        clf = LogisticRegression(multi_class='ovr')
        LRaccuracy, LRmarcof1 = k_fold_cv(combinedata, label, clf, k=10)
        print("LR Accuracy: ",LRaccuracy)
        print("LR F1: ", LRmarcof1)
        all_LR_accuracy.append(LRaccuracy)
        all_LR_f1.append(LRmarcof1)

# write evaluation result to excel
output = pd.DataFrame({'Name Group':allname,"Class number":num_class, "per_class_size":per_class_count,
                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

savePath = "../result/"+Dataset+"/meta_only/"
if not os.path.exists(savePath):
    os.makedirs(savePath)
filename = "2004_name_clf_meta=tf_threshold="+str(threshold)+".csv"
output.to_csv(savePath+filename, encoding='utf-8',index=False)
print("Done")

For name:  j_read
total sample size before apply threshold:  136
Counter({'0000-0002-5159-1192': 57, '0000-0002-9029-5185': 39, '0000-0002-9697-0962': 31, '0000-0002-4739-9245': 3, '0000-0003-0605-5259': 3, '0000-0003-4316-7006': 1, '0000-0002-0784-0091': 1, '0000-0002-3888-6631': 1})
['0000-0002-9697-0962', '0000-0002-9029-5185', '0000-0002-5159-1192']
Total sample size after apply threshold:  127
(0, 0)
(127, 79)
(127, 18)
(0, 0)
(0, 0)
2
(127, 97)
             precision    recall  f1-score   support

          0       0.54      0.48      0.51        31
          1       0.86      0.64      0.74        39
          2       0.67      0.82      0.74        57

avg / total       0.70      0.69      0.68       127

[15  1 15  6 25  8  7  3 47]
svc Accuracy:  0.6850393700787402
svc F1:  0.6613087247444019
             precision    recall  f1-score   support

          0       0.47      0.23      0.30        31
          1       0.77      0.59      0.67        39
          2       0.56    

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          1       0.71      1.00      0.83        69
          2       1.00      0.18      0.31        22

avg / total       0.70      0.72      0.63       101

[ 0 10  0  0 69  0  0 18  4]
LR Accuracy:  0.7227722772277227
LR F1:  0.3796725362990423
For name:  a_vega
total sample size before apply threshold:  20
Counter({'0000-0002-8207-9925': 10, '0000-0002-2178-2780': 8, '0000-0002-8148-5702': 1, '0000-0003-1082-0961': 1})
['0000-0002-8207-9925']
Total sample size after apply threshold:  10
For name:  k_smith
total sample size before apply threshold:  338
Counter({'0000-0002-6736-4779': 133, '0000-0001-8088-566X': 75, '0000-0002-1323-627X': 29, '0000-0002-8914-6457': 23, '0000-0001-6828-7480': 19, '0000-0001-8150-5702': 15, '0000-0003-2793-3460': 14, '0000-0002-4530-6914': 13, '0000-0003-2802-4939': 8, '0000-0002-0932-1412': 4, '0000-0002-2424-6254': 1, '0000-0001-6957-5361': 1,

             precision    recall  f1-score   support

          0       0.92      0.42      0.58        26
          1       1.00      0.07      0.13        14
          2       0.55      0.69      0.61        51
          3       0.67      0.85      0.75        79
          4       0.00      0.00      0.00        10
          5       0.74      0.85      0.80        82
          6       0.55      0.94      0.69       141
          7       0.00      0.00      0.00        14
          8       0.00      0.00      0.00        25
          9       1.00      0.25      0.40        28
         10       0.50      0.07      0.12        14
         11       1.00      0.55      0.71        20
         12       0.98      0.74      0.84        66
         13       1.00      0.33      0.50        15

avg / total       0.67      0.66      0.62       585

[ 11   0   4   4   0   0   7   0   0   0   0   0   0   0   0   1   6   5
   0   0   2   0   0   0   0   0   0   0   0   0  35   2   0   3  11   0
   

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        11
          1       1.00      0.21      0.35        14
          2       0.49      0.79      0.61       154
          3       0.25      0.05      0.08        22
          4       1.00      0.82      0.90        17
          5       0.00      0.00      0.00        11
          6       0.71      0.20      0.31        25
          7       0.65      0.94      0.77       211
          8       0.91      0.54      0.68        57
          9       0.76      0.57      0.65        28
         10       0.72      0.81      0.77       139
         11       0.97      0.67      0.79        45
         12       0.48      0.42      0.44        24
         13       0.85      0.51      0.64        57
         14       0.00      0.00      0.00        14
         15       0.68      0.47      0.56        55
         16       0.83      0.77      0.80        92
         17       0.67      0.12      0.20   

...

             precision    recall  f1-score   support

          0       0.59      0.38      0.47        26
          1       0.80      0.31      0.44        13
          2       0.59      0.83      0.69        42

avg / total       0.62      0.60      0.58        81

[10  0 16  1  4  8  6  1 35]
LR Accuracy:  0.6049382716049383
LR F1:  0.5342100101483016
For name:  m_bergman
total sample size before apply threshold:  36
Counter({'0000-0003-2589-2976': 20, '0000-0002-4529-4925': 13, '0000-0002-7033-5362': 2, '0000-0002-7018-1578': 1})
['0000-0002-4529-4925', '0000-0003-2589-2976']
Total sample size after apply threshold:  33
(0, 0)
(33, 15)
(33, 18)
(0, 0)
(0, 0)
2
(33, 33)
             precision    recall  f1-score   support

          0       0.91      0.77      0.83        13
          1       0.86      0.95      0.90        20

avg / total       0.88      0.88      0.88        33

[10  3  1 19]
svc Accuracy:  0.8787878787878788
svc F1:  0.8690476190476191
             precision    r

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        14
          1       1.00      0.45      0.62        11
          2       0.63      0.49      0.55        39
          3       1.00      0.33      0.50        21
          4       0.00      0.00      0.00        11
          5       0.97      0.67      0.80        58
          6       0.75      1.00      0.86       222

avg / total       0.75      0.77      0.73       376

[  0   0   3   0   0   0  11   0   5   1   0   0   0   5   0   0  19   0
   0   0  20   0   0   3   7   0   0  11   0   0   1   0   0   1   9   0
   0   2   0   0  39  17   0   0   1   0   0   0 221]
LR Accuracy:  0.773936170212766
LR F1:  0.4754617360449886
For name:  l_xiao
total sample size before apply threshold:  302
Counter({'0000-0001-8532-2727': 267, '0000-0002-4631-2443': 24, '0000-0003-0178-9384': 5, '0000-0003-4088-6101': 5, '0000-0002-0391-6909': 1})
['0000-0001-8532-2727', '0000-0002-4631-2443']
Total 

(0, 0)
(32, 20)
(32, 13)
(0, 0)
(0, 0)
2
(32, 33)
             precision    recall  f1-score   support

          0       1.00      0.70      0.82        10
          1       0.88      1.00      0.94        22

avg / total       0.92      0.91      0.90        32

[ 7  3  0 22]
svc Accuracy:  0.90625
svc F1:  0.8798498122653317
             precision    recall  f1-score   support

          0       1.00      0.40      0.57        10
          1       0.79      1.00      0.88        22

avg / total       0.85      0.81      0.78        32

[ 4  6  0 22]
LR Accuracy:  0.8125
LR F1:  0.7257142857142858
For name:  s_nielsen
total sample size before apply threshold:  290
Counter({'0000-0003-2417-0787': 108, '0000-0001-6391-7455': 72, '0000-0001-5341-1055': 44, '0000-0003-4175-3829': 21, '0000-0002-5777-6542': 13, '0000-0002-7780-7131': 11, '0000-0003-4309-5153': 7, '0000-0002-9754-0630': 4, '0000-0002-9214-2932': 4, '0000-0002-0458-3739': 3, '0000-0003-2770-9899': 1, '0000-0002-8449-476X': 

(0, 0)
(60, 36)
(60, 17)
(0, 0)
(0, 0)
2
(60, 53)
             precision    recall  f1-score   support

          0       1.00      0.86      0.92        21
          1       0.93      1.00      0.96        39

avg / total       0.95      0.95      0.95        60

[18  3  0 39]
svc Accuracy:  0.95
svc F1:  0.9430199430199431
             precision    recall  f1-score   support

          0       1.00      0.62      0.76        21
          1       0.83      1.00      0.91        39

avg / total       0.89      0.87      0.86        60

[13  8  0 39]
LR Accuracy:  0.8666666666666667
LR F1:  0.8358413132694938
For name:  r_mckay
total sample size before apply threshold:  53
Counter({'0000-0001-7781-1539': 31, '0000-0003-2723-5371': 17, '0000-0002-5602-6985': 4, '0000-0001-8042-2462': 1})
['0000-0001-7781-1539', '0000-0003-2723-5371']
Total sample size after apply threshold:  48
(0, 0)
(48, 30)
(48, 17)
(0, 0)
(0, 0)
2
(48, 47)
             precision    recall  f1-score   support

       

             precision    recall  f1-score   support

          0       0.90      1.00      0.95        43
          1       1.00      0.75      0.86        20

avg / total       0.93      0.92      0.92        63

[43  0  5 15]
LR Accuracy:  0.9206349206349206
LR F1:  0.901098901098901
For name:  c_peng
total sample size before apply threshold:  103
Counter({'0000-0003-3666-9833': 79, '0000-0003-3332-184X': 10, '0000-0001-7943-9873': 7, '0000-0001-6090-2944': 5, '0000-0002-0800-1417': 2})
['0000-0003-3666-9833', '0000-0003-3332-184X']
Total sample size after apply threshold:  89
(0, 0)
(89, 46)
(89, 22)
(0, 0)
(0, 0)
2
(89, 68)
             precision    recall  f1-score   support

          0       0.95      1.00      0.98        79
          1       1.00      0.60      0.75        10

avg / total       0.96      0.96      0.95        89

[79  0  4  6]
svc Accuracy:  0.9550561797752809
svc F1:  0.8626543209876543
             precision    recall  f1-score   support

          0       

In [35]:
# accuracy
from statistics import mean 
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

784
784
0.773022532793916
0.724474127951793


In [36]:
# f1
from statistics import mean 
# remove string from result
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

784
784
0.7093935712130214
0.5963653772711695
